<h1>120 Years of Olympic History (1896-2016)</h1>

Data source: https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results

The dataset contains information on Olympic athletes from the years 1896 to 2016. Such information includes their name, sex, age, height, weight, the Olympic Game, sport and event they participated in, and the medal they won.

This is a data cleaning project. I will clean it as if the data will be used for an exploratory data analysis project

In [1]:
# Load in some libraries (numpy might not be used)
import pandas as pd
import numpy as np

In [2]:
# Load in the dataset
data = pd.read_csv('athlete_events.csv')

# Output of the first five rows of data
data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


There are some missing values in the Height, Weight, and Medal columns which needs to be dealt with. The ID and Name columns do not seem to be useful, so they can be removed.

In [3]:
# Drop ID and Name columns
data.drop(['ID', 'Name'], axis=1, inplace=True)

In [4]:
# Before dealing with the missing values, get the descriptive statistics of the dataset to see if there are any abnormal values
data.describe()

,Age,Height,Weight,Year
count,261642.000000,210945.000000,208241.000000,271116.000000
mean,25.556898,175.338970,70.702393,1978.378480
std,6.393561,10.518462,14.348020,29.877632
min,10.000000,127.000000,25.000000,1896.000000
25%,21.000000,168.000000,60.000000,1960.000000
50%,24.000000,175.000000,70.000000,1988.000000
75%,28.000000,183.000000,79.000000,2002.000000
max,97.000000,226.000000,214.000000,2016.000000


No abnormal values were found. 

In [5]:
# Check for missing values and for the data type of each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Sex     271116 non-null  object 
 1   Age     261642 non-null  float64
 2   Height  210945 non-null  float64
 3   Weight  208241 non-null  float64
 4   Team    271116 non-null  object 
 5   NOC     271116 non-null  object 
 6   Games   271116 non-null  object 
 7   Year    271116 non-null  int64  
 8   Season  271116 non-null  object 
 9   City    271116 non-null  object 
 10  Sport   271116 non-null  object 
 11  Event   271116 non-null  object 
 12  Medal   39783 non-null   object 
dtypes: float64(3), int64(1), object(9)
memory usage: 26.9+ MB


In [6]:
# Replace missing values in the Medal column with '-'
data['Medal'].fillna('-', inplace=True)

In [7]:
# Find the skewness of Age for each Event
skewness = data.groupby('Event')['Age'].skew().to_frame('Age Skewness')
skewness.reset_index(inplace=True)
# Add the skewness of Age for each Event into our dataset
data = pd.merge(data, skewness, how='left', on='Event')

# Find the skewness of Height for each Event (since height and weight are correlated I'll assume the skewness for weight is about the same as height)
skewness2 = data.groupby('Event')['Height'].skew().to_frame('Height Skewness')
skewness2.reset_index(inplace=True)
# Add the skewness of Height for each Event into our dataset
data = pd.merge(data, skewness2, how='left', on='Event')

# Fill missing Age values with the median if it is skewed and with the mean if it is unskewed
data['Age'] = data['Age'].fillna(data.loc[abs(data['Age Skewness']) > 0.5].groupby('Event')['Age'].transform('median'))
data['Age'] = data['Age'].fillna(data.loc[abs(data['Age Skewness']) <= 0.5].groupby('Event')['Age'].transform('mean'))

# Fill missing Height and Weight values with the median if Height is skewed and with the mean if it is unskewed
data['Height'] = data['Height'].fillna(data.loc[abs(data['Height Skewness']) > 0.5].groupby('Event')['Height'].transform('median'))
data['Height'] = data['Height'].fillna(data.loc[abs(data['Height Skewness']) <= 0.5].groupby('Event')['Height'].transform('mean'))
data['Weight'] = data['Weight'].fillna(data.loc[abs(data['Height Skewness']) > 0.5].groupby('Event')['Weight'].transform('median'))
data['Weight'] = data['Weight'].fillna(data.loc[abs(data['Height Skewness']) <= 0.5].groupby('Event')['Weight'].transform('mean'))

# Another check for missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Sex              271116 non-null  object 
 1   Age              270958 non-null  float64
 2   Height           266371 non-null  float64
 3   Weight           264245 non-null  float64
 4   Team             271116 non-null  object 
 5   NOC              271116 non-null  object 
 6   Games            271116 non-null  object 
 7   Year             271116 non-null  int64  
 8   Season           271116 non-null  object 
 9   City             271116 non-null  object 
 10  Sport            271116 non-null  object 
 11  Event            271116 non-null  object 
 12  Medal            271116 non-null  object 
 13  Age Skewness     270933 non-null  float64
 14  Height Skewness  266261 non-null  float64
dtypes: float64(5), int64(1), object(9)
memory usage: 33.1+ MB


In [8]:
# Drop the skewness columns
data.drop(['Age Skewness', 'Height Skewness'],axis=1,inplace=True)

In [9]:
# Find the skewness of Age for each Sport
skewness = data.groupby('Sport')['Age'].skew().to_frame('Age Skewness')
skewness.reset_index(inplace=True)
# Add the skewness of Age for each Event into our dataset
data = pd.merge(data, skewness, how='left', on='Sport')

# Find the skewness of Height for each Sport (since height and weight are correlated I'll assume the skewness for weight is about the same as height)
skewness2 = data.groupby('Sport')['Height'].skew().to_frame('Height Skewness')
skewness2.reset_index(inplace=True)
# Add the skewness of Height for each Event into our dataset
data = pd.merge(data, skewness2, how='left', on='Sport')

# Fill missing Age values with the median if it is skewed and with the mean if it is unskewed
data['Age'] = data['Age'].fillna(data.loc[abs(data['Age Skewness']) > 0.5].groupby('Sport')['Age'].transform('median'))
data['Age'] = data['Age'].fillna(data.loc[abs(data['Age Skewness']) <= 0.5].groupby('Sport')['Age'].transform('mean'))

# Fill missing Height and Weight values with the median if Height is skewed and with the mean if it is unskewed
data['Height'] = data['Height'].fillna(data.loc[abs(data['Height Skewness']) > 0.5].groupby('Sport')['Height'].transform('median'))
data['Height'] = data['Height'].fillna(data.loc[abs(data['Height Skewness']) <= 0.5].groupby('Sport')['Height'].transform('mean'))
data['Weight'] = data['Weight'].fillna(data.loc[abs(data['Height Skewness']) > 0.5].groupby('Sport')['Weight'].transform('median'))
data['Weight'] = data['Weight'].fillna(data.loc[abs(data['Height Skewness']) <= 0.5].groupby('Sport')['Weight'].transform('mean'))

# Another check for missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Sex              271116 non-null  object 
 1   Age              271116 non-null  float64
 2   Height           270924 non-null  float64
 3   Weight           270830 non-null  float64
 4   Team             271116 non-null  object 
 5   NOC              271116 non-null  object 
 6   Games            271116 non-null  object 
 7   Year             271116 non-null  int64  
 8   Season           271116 non-null  object 
 9   City             271116 non-null  object 
 10  Sport            271116 non-null  object 
 11  Event            271116 non-null  object 
 12  Medal            271116 non-null  object 
 13  Age Skewness     271113 non-null  float64
 14  Height Skewness  270917 non-null  float64
dtypes: float64(5), int64(1), object(9)
memory usage: 33.1+ MB


In [10]:
# Drop the skewness columns
data.drop(['Age Skewness', 'Height Skewness'], axis=1, inplace=True)

In [11]:
# Find the skewness of Height for each Olympics Season (since height and weight are correlated I'll assume the skewness for weight is about the same as height)
skewness = data.groupby('Season')['Height'].skew().to_frame('Height Skewness')
skewness.reset_index(inplace=True)
# Add the skewness of Height for each Event into our dataset
data = pd.merge(data, skewness, how='left', on='Season')

# Fill missing Height and Weight values with the median if Height is skewed and with the mean if it is unskewed
data['Height'] = data['Height'].fillna(data.loc[abs(data['Height Skewness']) > 0.5].groupby('Season')['Height'].transform('median'))
data['Height'] = data['Height'].fillna(data.loc[abs(data['Height Skewness']) <= 0.5].groupby('Season')['Height'].transform('mean'))
data['Weight'] = data['Weight'].fillna(data.loc[abs(data['Height Skewness']) > 0.5].groupby('Season')['Weight'].transform('median'))
data['Weight'] = data['Weight'].fillna(data.loc[abs(data['Height Skewness']) <= 0.5].groupby('Season')['Weight'].transform('mean'))

# Another check for missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 0 to 271115
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Sex              271116 non-null  object 
 1   Age              271116 non-null  float64
 2   Height           271116 non-null  float64
 3   Weight           271116 non-null  float64
 4   Team             271116 non-null  object 
 5   NOC              271116 non-null  object 
 6   Games            271116 non-null  object 
 7   Year             271116 non-null  int64  
 8   Season           271116 non-null  object 
 9   City             271116 non-null  object 
 10  Sport            271116 non-null  object 
 11  Event            271116 non-null  object 
 12  Medal            271116 non-null  object 
 13  Height Skewness  271116 non-null  float64
dtypes: float64(4), int64(1), object(9)
memory usage: 31.0+ MB


Finally, all the missing values are removed. On a side note, there turned out to be some redundant code, so if I were to redo this project, I would write a function to shorten the code.

In [12]:
# Drop the skewness columns
data.drop('Height Skewness', axis=1, inplace=True)

In [13]:
# Split the data into Winter Olympics and Summer Olympics
summer_olympics = data.loc[data['Season'] == 'Summer']
winter_olympics = data.loc[data['Season'] == 'Winter']

In [14]:
# First five rows of the cleaned dataset (summer)
summer_olympics.head()

,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,M,24.0,180.000000,80.000000,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,-
1,M,23.0,170.000000,60.000000,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,-
2,M,24.0,177.480339,73.086644,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,-
3,M,34.0,182.480000,95.615385,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
26,F,18.0,168.000000,57.485390,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,-


In [15]:
# First five rows of the cleaned dataset (winter)
winter_olympics.head()

,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
4,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,-
5,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",-
6,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,-
7,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",-
8,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,-


Now the the dataset is ready for exploring!